In [6]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import os

load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
PATH_TO_CERT = os.getenv('PATH_TO_CERT')

connection_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(
    connection_str,
    connect_args={
        "sslmode": "verify-full",
        "sslrootcert": PATH_TO_CERT,
        "target_session_attrs": "read-write"
    }
)

In [11]:
create_table_query = """
drop table if exists posts;
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_table_query))
except Exception as e:
    print("Ошибка:", e)


In [4]:
with engine.connect() as conn:
    df = pd.read_sql('''
select * from posts
    ''', conn)

df

,id,text,created_at


In [73]:
def strip_edges_allow_punct(s: str):
    allowed_punct = set(".,!?;:-–—")  # можно расширять

    # Левый указатель — пока не буква/цифра
    left = 0
    while left < len(s) and not s[left].isalnum():
        left += 1

    # Правый указатель — пока не буква/цифра/пунктуация
    right = len(s) - 1
    while right >= 0 and not (s[right].isalnum() or s[right] in allowed_punct):
        right -= 1

    # Если всё мусор
    if right < left:
        return ""

    return s[left:right+1]



def process_str(s):
    s = "\n".join(strip_edges_allow_punct(p) for p in s.split("\n") if p)
    
    for suf in [
        "Слушать прямой эфир",
        "Читать РБК Стиль в Telegram",
        "РБК Events, 18",
        "Подписаться | Онлайн-сомелье",
        "Читать РБК в Telegram",
        "Следить за новостями РБК в Telegram",
        "Следить за новостями РБК в МАХ",
        "Другие видео этого дня — в телеграм-канале РБК",
        "РБК в Telegram и MAX",
        "РБК в Telegram | MAX",
        "Подписаться на «РБК Спорт",
        "Картина дня — в телеграм-канале РБК",
        "Самые важные новости — в канале РБК в МАХ",
        "Больше инфографики — в телеграм-канале РБК",
        "Подписаться на «Сам ты инвестор!",
        "Читать РБК Недвижимость в Telegram"
    ]:
        s = s.removesuffix(suf).strip()

    parts = [p for p in s.split("\n") if p]
    
    prev_parts = [0] * 1000
    while len(prev_parts) != len(parts) and len(parts) != 0:
        prev_parts = parts
        if "Фото:" in parts[-1] or "Данные:" in parts[-1]:
            parts = parts[:-1]
    
    return "\n".join(parts)

import pandas as pd

rbc = pd.read_csv("src/dataset/rbc/channel_rbc_news_posts.csv")
rbc["message_dt"] = pd.to_datetime(rbc["message_dt"]).dt.date
rbc = rbc[["message_id", "channel_id", "message_dt", "views", "content"]].astype({"views": int}).sample(len(rbc))
rbc["content"] = rbc["content"].apply(lambda x: process_str(x))
rbc = rbc[rbc["content"].apply(
        lambda x: not any(v in [p for p in x.split("\n") if p][-1] for v in ["Реклама.", "Реклама,"]) if x else False
    )]
rbc

,message_id,channel_id,message_dt,views,content
4498,130471,rbc_news,2025-09-12,139946,Адвокат бывшего мэра Стамбула Экрема Имамоглу ...
1054,134025,rbc_news,2025-10-22,112469,Госдума приняла в первом чтении проект бюджета...
283,136520,rbc_news,2025-11-24,118238,Силы ПВО в период с 14:00 до 20:00 мск уничтож...
4406,124142,rbc_news,2025-07-15,122075,В Госдуме из-за мощных ливней затопило курилку...
1378,135060,rbc_news,2025-11-05,114842,Американская актриса и посол доброй воли ЮНИСЕ...
...,...,...,...,...,...
1467,130799,rbc_news,2025-09-17,135139,"Кинокомпании Walt Disney, Universal и Warner B..."
4581,123624,rbc_news,2025-07-10,121628,Главные новости к утру — на телеканале РБК
4281,130775,rbc_news,2025-09-16,120652,Путин примерил тепловизионные очки «Стрекоза» ...
241,136585,rbc_news,2025-11-25,101214,Российского посла вызвали в МИД Молдавии из-за...


In [74]:
create_table_query = """
drop table if exists posts;
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_table_query))
except Exception as e:
    print("Ошибка:", e)

rbc.to_sql('posts', engine, if_exists='replace', index=False)

800

In [22]:
with engine.connect() as conn:
    df = pd.read_sql('''
select * from posts
    ''', conn)

df

,message_id,channel_id,message_dt,views,content
0,137228,rbc_news,2025-12-03,40045,Суд признал писателя Бориса Акунина (настоящее...
1,137226,rbc_news,2025-12-03,53463,"На характере переговоров с США, прошедших нака..."
2,137224,rbc_news,2025-12-03,56667,Еврокомиссия намерена запретить исполнение вну...
3,137223,rbc_news,2025-12-03,55604,🎙 Прямо сейчас в эфире Радио РБК обсуждаем пла...
4,137222,rbc_news,2025-12-03,61798,Коллегия Еврокомиссии одобрила «потенциальный ...
...,...,...,...,...,...
4820,116046,rbc_news,2025-04-15,106022,"Репортаж телеканала РБК из Курска, который ноч..."
4821,116045,rbc_news,2025-04-15,108803,Суд на Сахалине вынес первое решение по делу о...
4822,116044,rbc_news,2025-04-15,108074,Оператор национальных лотерей Франции стал отв...
4823,116041,rbc_news,2025-04-15,125020,Аэропорт южнокорейского города Муан регулярно ...


In [40]:
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
import numpy as np
import os

load_dotenv()

QDRANT_URL = os.getenv('QDRANT_URL')

client = QdrantClient(
    url=QDRANT_URL,
)

In [2]:
client.create_collection(
    collection_name="my_collection",
    vectors_config=models.VectorParams(
        size=384, 
        distance=models.Distance.COSINE
    )
)


True

In [ ]:
points = [
    models.PointStruct(
        id=1,
        vector=np.ones(384).tolist(),
        payload={"text": "Пример документа 1"}
    ),
    models.PointStruct(
        id=2,
        vector=(-np.ones(384)).tolist(),
        payload={"text": "Пример документа 2"}
    )
]

client.upsert(
    collection_name="my_collection",
    points=points,
    wait=True
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [42]:
results = client.query_points(
    collection_name="my_collection",
    query=(-np.ones(384)).tolist(),
    limit=1
)

In [43]:
results.points[0]

ScoredPoint(id=2, version=3, score=0.9999998, payload={'text': 'Пример документа 2'}, vector=None, shard_key=None, order_value=None)